## Preprocessing

In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

# Variables
model_results = []

import warnings
warnings.filterwarnings('ignore')  # Ignore all warnings

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(["EIN", "NAME"], axis=1)
application_df.head(5)


In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

In [ ]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
counts = application_df["APPLICATION_TYPE"].value_counts()
counts

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = counts[counts<500].index.tolist()


# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure replacement was successful
application_df['APPLICATION_TYPE'].value_counts()

In [ ]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
counts = application_df["CLASSIFICATION"].value_counts()
counts


In [ ]:
# You may find it helpful to look at CLASSIFICATION value counts >1
counts[counts > 1]

In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = counts[counts < 1500].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

In [ ]:
# Convert categorical data to numeric with `pd.get_dummies`
categorical_columns = application_df.select_dtypes(include=['object']).columns
application_df = pd.get_dummies(application_df, columns=categorical_columns)
application_df.head(5)


In [ ]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"].values
X = application_df.drop(["IS_SUCCESSFUL"], axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

## Base Model
Model based on starter code outputs.

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn_base = tf.keras.models.Sequential()

# First hidden layer
nn_base.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_base.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_base.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_base.summary()

In [ ]:
# Compile the model
nn_base.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
base_model = nn_base.fit(X_train_scaled,y_train,epochs=100)

In [ ]:
# Plot Results
training_df = pd.DataFrame(base_model.history, index=range(1,len(base_model.history["loss"])+1))
training_df.plot(y="loss")
training_df.plot(y="accuracy")


# Evaluate the model using the test data
model_loss, model_accuracy = nn_base.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#  Add results to a dictionary for comparison.
model_info = {
    'model_name': 'Base',
    'loss': model_loss,
    'accuracy': model_accuracy
}

model_results.append(model_info)


## Optimization Attempt 1
* **Changes:**
  * Increase number of epochs from 100 to 200


In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn_optomize_v1 = tf.keras.models.Sequential()

# First hidden layer
nn_optomize_v1.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_optomize_v1.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn_optomize_v1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_optomize_v1.summary()

In [ ]:
# Compile the model
nn_optomize_v1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
model_optomize_v1 = nn_optomize_v1.fit(X_train_scaled,y_train,epochs=200)

In [ ]:
# Plot Results
training_df = pd.DataFrame(model_optomize_v1.history, index=range(1,len(model_optomize_v1.history["loss"])+1))
training_df.plot(y="loss")
training_df.plot(y="accuracy")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_optomize_v1.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#  Add results to a dictionary for comparison.
model_info = {
    'model_name': 'Optimization Attempt 1',
    'loss': model_loss,
    'accuracy': model_accuracy
}

model_results.append(model_info)

## Optimization Attempt 2
* **Changes:**
  * Increased nodes in layer one to 3 times the number of columns in dataset.
  * Increased nodes in layer two to 1/2 the number of nodes as layer one

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  132
hidden_nodes_layer2 = 66


nn_optomize_v2 = tf.keras.models.Sequential()

# First hidden layer
nn_optomize_v2.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_optomize_v2.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))


# Output layer
nn_optomize_v2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_optomize_v2.summary()

In [ ]:
# Compile the model
nn_optomize_v2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
model_optomize_v2 = nn_optomize_v2.fit(X_train_scaled,y_train,epochs=100)

In [ ]:
# Plot Results
training_df = pd.DataFrame(model_optomize_v2.history, index=range(1,len(model_optomize_v2.history["loss"])+1))
training_df.plot(y="loss")
training_df.plot(y="accuracy")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_optomize_v2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#  Add results to a dictionary for comparison.
model_info = {
    'model_name': 'Optimization Attempt 2',
    'loss': model_loss,
    'accuracy': model_accuracy
}

model_results.append(model_info)

# Optimization Attempt 3
* **Changes:**
  * Changed the number of nodes in layer one to 120
  * Changed the number of nodes in layer two to 90.
  * Added a third layer with 30 nodes.

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  120
hidden_nodes_layer2 = 90
hidden_nodes_layer3 = 30

nn_optomize_v3 = tf.keras.models.Sequential()

# First hidden layer
nn_optomize_v3.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn_optomize_v3.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn_optomize_v3.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn_optomize_v3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_optomize_v3.summary()

In [ ]:
# Compile the model
nn_optomize_v3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
model_optomize_v3 = nn_optomize_v3.fit(X_train_scaled,y_train,epochs=100)

In [ ]:
# Plot Results
training_df = pd.DataFrame(model_optomize_v3.history, index=range(1,len(model_optomize_v3.history["loss"])+1))
training_df.plot(y="loss")
training_df.plot(y="accuracy")

# Evaluate the model using the test data
model_loss, model_accuracy = nn_optomize_v3.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

#  Add results to a dictionary for comparison.
model_info = {
    'model_name': 'Optimization Attempt 3',
    'loss': model_loss,
    'accuracy': model_accuracy
}

model_results.append(model_info)

### KerasTuner Library
I decided to run the Keras Tuner Library to see if can improve the results and see what it suggests for model perameters.

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=43))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])

    return nn_model

In [ ]:
# Installs keras-tuner on Google Colab
# !pip install keras-tuner

In [ ]:
# Import the kerastuner library
import keras_tuner as kt

In [ ]:
tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    hyperband_iterations=2)

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

### Export Model
I output a report on the models I created with different iterations.  Based on their results, I chose my initial *Base* model as the most optimized since it had a slightly higher accuracy of .73 and a lower loss of .56.

In [ ]:
# Display the results for my different models for comparison. 
print("================================================================================") 
print("|                              Model Results                                   |")
print("================================================================================")
print()
for result in model_results:
    print(f"{result['model_name']}: Loss: {result['loss']}, Accuracy: {result['accuracy']}")

In [ ]:
# Export our model to HDF5 file
nn_base.save("AlphabetSoupCharity_Optimization.h5")